In [193]:
import pandas as pd
from collections import Counter  
import json
import numpy as np
from sqlalchemy import create_engine

In [103]:
tweets_df = pd.read_csv('twitter_dataset.csv')

In [104]:
tweets_df.head()

,Unnamed: 0,hashtags,tweet,user_name,location
0,0,[],RT @QasimRashid: I’m an American Muslim human ...,pghcovert,the steel city
1,1,[],"This is a morbid article, but thanks.\n\nInvas...",canucklaw1,"British Columbia, Canada"
2,2,[],RT @BillKristol: I’ve been wondering why the v...,Miss_de_Haro,NaN
3,3,[],RT @DVATW: Oh look! Calais gets even more enri...,The_Nitr0,NaN
4,4,[],RT @QasimRashid: I’m an American Muslim human ...,LunaticEuphoric,"Lahore, Pakistan"


# Data wrangling

In [105]:
tweets_df = tweets_df.dropna()

In [106]:
def lowercase_locations(loc):
    return loc.lower()
    

In [107]:
tweets_df['location'] = tweets_df['location'].apply(lowercase_locations)

In [108]:
def fix_names(loc):
    
    if 'turkey' in loc:
        return 'turkey'
    
    elif 'türkiye' in loc:
        return 'turkey'
    
    elif 'sakarya' in loc:
        return 'turkey'
    
    elif 'deutschland' in loc:
        return 'deutschland'
    
    elif 'england' in loc:
        return 'united kingdom'
    
    elif 'london' in loc:
        return 'united kingdom'
    
    elif 'united kingdom' in loc:
        return 'united kingdom'
    
    elif 'british columbia' in loc:
        return 'canada'
    
    elif 'ontario' in loc:
        return 'canada'
    
    elif 'calgary' in loc:
        return 'canada'
    
    elif 'mexico' in loc:
        return 'mexico'
    elif 'pakistan' in loc:
        return 'pakistan'
    
    return loc

In [109]:
tweets_df['location'] = tweets_df['location'].apply(fix_names)

In [111]:
tweets_df = tweets_df.drop('Unnamed: 0', axis =1)

In [112]:
import json

with open('states_hash.json') as json_file:  
    data = json.load(json_file)

In [113]:
us_states = [state.lower() for state in data.values()]

In [114]:
def fixed_usa(loc):
    for i in range(len(us_states)):
        if us_states[i] in loc:
            return us_states[i]
    return loc
    

In [115]:
tweets_df['location'] = tweets_df['location'].apply(fixed_usa)

In [116]:
countries = ['turkey','deutschland','united kingdom','canada','germany', 'mexico', 'pakistan']
    
def eliminating_locations(loc):
    if (loc in countries):
        return loc
    elif loc in us_states:
        return loc
    

In [117]:
tweets_df['location'] = tweets_df['location'].map(eliminating_locations)

In [118]:
tweets_df.head()

,hashtags,tweet,user_name,location
0,[],RT @QasimRashid: I’m an American Muslim human ...,pghcovert,None
1,[],"This is a morbid article, but thanks.\n\nInvas...",canucklaw1,canada
4,[],RT @QasimRashid: I’m an American Muslim human ...,LunaticEuphoric,pakistan
6,[],RT @Gracie5111: The EU dumped millions of Musl...,CuriousLor,canada
7,[],RT @BillKristol: I’ve been wondering why the v...,MelindaPowell,texas


In [119]:
tweets_df = tweets_df.dropna().reset_index(drop=True)

In [190]:
tweets_df.head()

,hashtags,tweet,user_name,location
11,['ICEraids'],RT @Harryslaststand: Tomorrow #ICEraids will b...,WinstonOrozco_1,new york
17,"['Christians', 'Evangelicals', 'Trump', 'Jesus']",RT @mog7546: SEPARATE CHILDREN FROM THEIR PARE...,SarahMeinzer,texas
23,"['Christians', 'Evangelicals', 'Trump', 'Jesus']",RT @mog7546: SEPARATE CHILDREN FROM THEIR PARE...,pushandpulljlm,north carolina
79,['Brexit'],Maybe if the same waste of energy and effort t...,dre_65,united kingdom
105,['ClimateAction'],RT @UN: Refugees are joining other community m...,JamesGi27668130,florida


In [128]:
def eliminating_null_hashtags(ht):
    if ht == '[]':
        return np.nan
    else:
        return ht
    

In [130]:
tweets_df['hashtags'] = tweets_df['hashtags'].map(eliminating_null_hashtags)

In [133]:
tweets_df = tweets_df.dropna()

In [134]:
tweets_df.head()

,hashtags,tweet,user_name,location
11,['ICEraids'],RT @Harryslaststand: Tomorrow #ICEraids will b...,WinstonOrozco_1,new york
17,"['Christians', 'Evangelicals', 'Trump', 'Jesus']",RT @mog7546: SEPARATE CHILDREN FROM THEIR PARE...,SarahMeinzer,texas
23,"['Christians', 'Evangelicals', 'Trump', 'Jesus']",RT @mog7546: SEPARATE CHILDREN FROM THEIR PARE...,pushandpulljlm,north carolina
79,['Brexit'],Maybe if the same waste of energy and effort t...,dre_65,united kingdom
105,['ClimateAction'],RT @UN: Refugees are joining other community m...,JamesGi27668130,florida


In [149]:
tweets_refugees = tweets_df[tweets_df['hashtags'].str.contains('efu')]

In [189]:
tweets_refugees.head()

,hashtags,tweet,user_name
location,,,
turkey,71,71,71
united kingdom,19,19,19
canada,11,11,11
new york,6,6,6
washington,5,5,5


In [158]:
tweets_refugees = tweets_refugees.groupby('location').count().sort_values('hashtags', ascending= False)

In [169]:
tweets_refugees_filter = tweets_refugees[tweets_refugees['hashtags'] >4]

In [170]:
tweets_refugees_filter = tweets_refugees_filter.drop(['tweet' , 'user_name'], axis =1)


In [176]:
tweets_refugees_filter.columns = ['#refugee_total_counts']

In [178]:
population = [79.81, 66.04, 37.06, 8.623, 7.536]

In [179]:
tweets_refugees_filter['population'] = population

In [181]:
tweets_refugees_filter['ratio_per_100k'] = tweets_refugees_filter['#refugee_total_counts'] / tweets_refugees_filter['population']

In [185]:
tweets_refugees_filter = tweets_refugees_filter.sort_values('ratio_per_100k', ascending=False)

In [195]:
tweets_refugees_filter.columns = ['ht_refugee_total_count','population','ratio_per_M']

In [199]:
tweets_refugees_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, turkey to united kingdom
Data columns (total 3 columns):
ht_refugee_total_count    5 non-null int64
population                5 non-null float64
ratio_per_M               5 non-null float64
dtypes: float64(2), int64(1)
memory usage: 160.0+ bytes


In [200]:
tweets_refugees_filter

,ht_refugee_total_count,population,ratio_per_M
location,,,
turkey,71,79.810,0.889613
new york,6,8.623,0.695814
washington,5,7.536,0.663482
canada,11,37.060,0.296816
united kingdom,19,66.040,0.287704


In [197]:
tweets_refugees_filter.to_csv('ht_refugees_locations.csv')

In [191]:
driver = 'mysql+pymysql'
user = 'virdishc_12345'
password = '12345'
host = 'az1-ss5.a2hosting.com'
database = 'virdishc_IH_Project3'

In [ ]:
connection_string = f'{driver}://{user}:{password}@{host}/{database}'
engine= create_engine(connection_string)

In [ ]:
tweets_refugees_filter.to_sql('ht_refugees_locations',engine)